In [ ]:
#   Author: Alexis Hilts
#   Date:   23rd April, 2025

#   Script Purpose: This script quantifies the neurite density in images taken of dissociated cortical cells by
#   subtracting the mask image from the original image. The neurite density will be output as an array which can be plotted.                   

# import necessary libraries
import os
import numpy as np
import cv2
from tifffile import imread, imwrite
from matplotlib import pyplot as plt

# Folder containing images and masks
folder_path = "YOUR_IMAGE_FOLDER_PATH_HERE"

# Sort images and masks according to filenames
image_files = sorted([f for f in os.listdir(folder_path) if f.endswith("im.tiff")])
mask_files = sorted([f for f in os.listdir(folder_path) if f.endswith("mask.tif")])

# Initialize list to store neurite densities - change name of list for each condition you are running
density_condition_list = []

# Process each image and mask pair
for img_file, mask_file in zip(image_files, mask_files):
    # Load image and mask
    img_path = os.path.join(folder_path, img_file)
    mask_path = os.path.join(folder_path, mask_file)
    
    # Select channel 2 (AF647 channel)
    neur_im = imread(img_path)
    neur_im = neur_im[:,:,1]
    mask_im = imread(mask_path)
    mask_im = mask_im[:,:,1]

    # Apply the mask: subtract the mask from the image to isolate the neurites
    masked_image = neur_im - mask_im
    masked_image[masked_image < 0] = 0  # Clip negative values to 0

    # Binarize the masked image (using Otsu's method)
    _, binary_img = cv2.threshold(masked_image.astype(np.uint16), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Save binary image as a .tif file
    binary_filename = f"{os.path.splitext(img_file)[0]}_binary.tif"
    binary_filepath = os.path.join(folder_path, binary_filename)
    imwrite(binary_filepath, binary_img.astype(np.uint8) * 255)  # Scale binary to 8-bit (0-255)
    print(f"Saved binary image: {binary_filepath}")
   
    # Calculate neurite density
    neurite_density = np.sum(binary_img > 0) / binary_img.size
    density_condition_list.append(neurite_density) # append image neurite density to list

    # Plot the results
    fig, axs = plt.subplots(1, 4, figsize=(15, 5))

    axs[0].imshow(neur_im, cmap='gray')
    axs[0].set_title(f'Image: {img_file}')
    axs[0].axis('off')

    axs[1].imshow(mask_im, cmap='gray')
    axs[1].set_title(f'Mask: {mask_file}')
    axs[1].axis('off')
    
    axs[2].imshow(masked_image, cmap='gray')
    axs[2].set_title('Masked Image')
    axs[2].axis('off')

    axs[3].imshow(binary_img, cmap='gray')
    axs[3].set_title('Binarized Masked Image')
    axs[3].axis('off')

    plt.tight_layout()
    plt.show()

# Print and optionally save the calculated densities
print("Neurite Densities:", density_condition_list)


In [ ]:
# save the list for each condition (update this as you run each folder - there is definitely a better way to do this)
drg_neurite_densities_c = [drg_c]
drg_neurite_densities_0125 = [drg_0125]
drg_neurite_densities_025 = [drg_025]
drg_neurite_densities_05 = [drg_05]
drg_neurite_densities_1 = [drg_1]
drg_neurite_densities_2 = [drg_2]

In [ ]:
# Plot as points with error bars

# Combine all conditions into a list
all_conditions = [
    drg_neurite_densities_c,
    drg_neurite_densities_0125,
    drg_neurite_densities_025,
    drg_neurite_densities_05,
    drg_neurite_densities_1,
    drg_neurite_densities_2
]

# Labels for the conditions
condition_labels = [
    "Base Media",
    "0.125 nM Epo-B",
    "0.25 nM Epo-B",
    "0.5 nM Epo-B",
    "1 nM Epo-B",
    "2 nM Epo-B"
]

# Calculate means and standard deviations for each condition
means = [np.mean(condition) for condition in all_conditions]
stds = [np.std(condition) for condition in all_conditions]

# Plot the means with standard deviations as error bars
fig, ax = plt.subplots(figsize=(8, 6), dpi=120)
x = np.arange(len(condition_labels))  # x positions for the conditions
ax.errorbar(x, means, yerr=stds, fmt='o', capsize=5, color='blue', ecolor='black', label='Neurite Density')


# Add labels and title
ax.set_xticks(x)
ax.set_xticklabels(condition_labels, rotation=15, ha='right')
ax.set_ylabel('Neurite Density')
ax.set_xlabel('Condition')
ax.set_title('Neurite Density Across Epo-B Conditions')
#ax.legend('Average Neurite Density', loc = 2, prop={'size': 5})
plt.tight_layout()
#plt.legend()

# Show the plot
plt.show()

In [ ]:
# Plot as boxplot

all_conditions = [
    np.ravel(drg_neurite_densities_c),
    np.ravel(drg_neurite_densities_0125),
    np.ravel(drg_neurite_densities_025),
    np.ravel(drg_neurite_densities_05),
    np.ravel(drg_neurite_densities_1),
    np.ravel(drg_neurite_densities_2)
]
print(all_conditions)

# Labels for the conditions
condition_labels = [
    "Base Media",
    "0.125 nM Epo-B",
    "0.25 nM Epo-B",
    "0.5 nM Epo-B",
    "1 nM Epo-B",
    "2 nM Epo-B"
]

# Create a boxplot
fig, ax = plt.subplots(figsize=(8, 6), dpi=120)
ax.boxplot(all_conditions, labels=condition_labels, patch_artist=True, boxprops=dict(facecolor='lightblue', color='black'), 
           medianprops=dict(color='red'), whiskerprops=dict(color='black'), capprops=dict(color='black'))

# Add labels and title
ax.set_ylabel('Neurite Density')
ax.set_xlabel('Condition')
ax.set_title('Neurite Density Across Epo-B Conditions')
plt.xticks(rotation=15, ha='right')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Run t-test on all conditions

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import pandas as pd

# Combine all conditions into a list
all_conditions = [
    neurite_densities_m,
    neurite_densities_0125,
    neurite_densities_025,
    neurite_densities_05,
    neurite_densities_1,
]

# Labels for the conditions
condition_labels = [
    "Base Media",
    "0.125 nM Epo-B",
    "0.25 nM Epo-B",
    "0.5 nM Epo-B",
    "1 nM Epo-B",
]

# Calculate means and standard deviations for each condition
means = [np.mean(condition) for condition in all_conditions]
stds = [np.std(condition) for condition in all_conditions]

# Plot the means with standard deviations as error bars
fig, ax = plt.subplots(figsize=(8, 6), dpi=120)
x = np.arange(len(condition_labels))  # x positions for the conditions
ax.errorbar(x, means, yerr=stds, fmt='o', capsize=5, color='blue', ecolor='black', label='Neurite Density')

# Add labels and title
ax.set_xticks(x)
ax.set_xticklabels(condition_labels, rotation=15, ha='right')
ax.set_ylabel('Neurite Density')
ax.set_xlabel('Condition')
ax.set_title('Neurite Density Across Conditions with Epo-B')

# Perform pairwise t-tests and add significance annotations
for i in range(len(all_conditions)):
    for j in range(i + 1, len(all_conditions)):
        # Flatten the arrays for t-test compatibility
        condition_i = np.array(all_conditions[i]).flatten()
        condition_j = np.array(all_conditions[j]).flatten()
        
        # Perform a t-test between condition i and condition j
        t_stat, p_val = ttest_ind(condition_i, condition_j)

        # Determine the y position for the significance bar
        y = max(means[i] + stds[i], means[j] + stds[j]) + 0.02
        bar_height = 0.01
        star_height = 0.01

        # Add significance annotation
        if p_val < 0.05:
            ax.plot([x[i], x[j]], [y-0.015, y-0.015], lw=1.5, color='black')  # Add the significance line
            ax.text((x[i] + x[j]) * 0.5, y-0.015, '*', ha='center', va='bottom', color='black')  # Add star

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

